<a href="https://colab.research.google.com/github/BossDevinator/IS_4482/blob/main/A3_Moore_Devan.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Assignment 3: Decision Tree Classification and Evaluation
###Author: Devan Moore
9/4/2024

#Chunk 1:
###Set up, data import and inspection code for the following

In [1]:
import pandas as pd
import sklearn
from sklearn import tree
from sklearn import metrics
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import numpy as np

In [3]:
df = pd.read_csv('https://raw.githubusercontent.com/matthewpecsok/4482_fall_2024/main/data/CD_additional_balanced.csv')

In [20]:
df.head()

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,duration,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed
0,41,blue-collar,divorced,basic.4y,unknown,yes,no,telephone,may,mon,1575,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0
1,49,entrepreneur,married,university.degree,unknown,yes,no,telephone,may,mon,1042,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0
2,49,technician,married,basic.9y,no,no,no,telephone,may,mon,1467,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0
3,41,technician,married,professional.course,unknown,yes,no,telephone,may,mon,579,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0
4,45,blue-collar,married,basic.9y,unknown,yes,no,telephone,may,mon,461,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9280 entries, 0 to 9279
Data columns (total 21 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   age             9280 non-null   int64  
 1   job             9280 non-null   object 
 2   marital         9280 non-null   object 
 3   education       9280 non-null   object 
 4   default         9280 non-null   object 
 5   housing         9280 non-null   object 
 6   loan            9280 non-null   object 
 7   contact         9280 non-null   object 
 8   month           9280 non-null   object 
 9   day_of_week     9280 non-null   object 
 10  duration        9280 non-null   int64  
 11  campaign        9280 non-null   int64  
 12  pdays           9280 non-null   int64  
 13  previous        9280 non-null   int64  
 14  poutcome        9280 non-null   object 
 15  emp.var.rate    9280 non-null   float64
 16  cons.price.idx  9280 non-null   float64
 17  cons.conf.idx   9280 non-null   f

In [6]:
df.describe()

,age,duration,campaign,pdays,previous,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed
count,9280.000000,9280.000000,9280.000000,9280.000000,9280.000000,9280.000000,9280.000000,9280.000000,9280.000000,9280.000000
mean,40.403448,387.361746,2.333297,887.282435,0.315302,-0.496272,93.479178,-40.218125,2.960209,5135.306487
std,12.062030,357.711742,2.334467,313.802415,0.700671,1.721204,0.634510,5.360642,1.890402,87.105317
min,17.000000,1.000000,1.000000,0.000000,0.000000,-3.400000,92.201000,-50.800000,0.634000,4963.600000
25%,31.000000,145.000000,1.000000,999.000000,0.000000,-1.800000,92.893000,-42.700000,1.244000,5076.200000
50%,38.000000,265.000000,2.000000,999.000000,0.000000,-0.100000,93.444000,-41.800000,4.021000,5191.000000
75%,48.000000,528.000000,3.000000,999.000000,0.000000,1.400000,93.994000,-36.400000,4.959000,5228.100000
max,98.000000,4199.000000,39.000000,999.000000,6.000000,1.400000,94.767000,-26.900000,5.045000,5228.100000


In [7]:
df.shape

(9280, 21)

In [8]:
df.isnull().sum()

,0
age,0
job,0
marital,0
education,0
default,0
housing,0
loan,0
contact,0
month,0
day_of_week,0


#Chunk 2:
###Target Variable

In [11]:
y_count = df.y.value_counts()
y_proportion = df.y.value_counts(normalize=True)
print(y_count)
print(y_proportion)

y
yes    4640
no     4640
Name: count, dtype: int64
y
yes    0.5
no     0.5
Name: proportion, dtype: float64


This target variable is perfectly split between 4,640 'yes' and 4,640 'no'. It is balanced at 50% for both.

In [15]:
y_target = df.pop('y')

KeyError: 'y'

#Chunk 3:
###Data Preparation

Within this data set the following features are categorical:


*   Marital
*   Job
*   Education
*   Default
*   Housing
*   Loan
*   Month
*   Day of week
*   Contact
*   Campaign
*   Previous
*   Poutcome

And these are quantitative:

*   age
*   duration
*   pdays
*   emp.var.rate
*   cons.price.idx
*   cons.conf.idx
*   eurabor3m
*   nr.employed



In [22]:
marital_count = df.marital.value_counts()
print(marital_count)
job_count = df.job.value_counts()
print(job_count)
education_count = df.education.value_counts()
print(education_count)
default_count = df.default.value_counts()
print(default_count)
housing_count = df.housing.value_counts()
print(housing_count)
loan_count = df.loan.value_counts()
print(loan_count)
month_count = df.month.value_counts()
print(month_count)
day_count = df.day_of_week.value_counts()
print(day_count)
contact_count = df.contact.value_counts()
print(contact_count)
campaign_count = df.campaign.value_counts()
print(campaign_count)
previous_count = df.previous.value_counts()
print(previous_count)
poutcome_count = df.poutcome.value_counts()
print(poutcome_count)

marital
married     5338
single      2900
divorced    1021
unknown       21
Name: count, dtype: int64
job
admin.           2517
blue-collar      1769
technician       1459
services          773
management        651
retired           595
student           358
entrepreneur      308
self-employed     306
unemployed        248
housemaid         216
unknown            80
Name: count, dtype: int64
education
university.degree      3007
high.school            2102
professional.course    1190
basic.9y               1177
basic.4y                895
basic.6y                458
unknown                 445
illiterate                6
Name: count, dtype: int64
default
no         7824
unknown    1456
Name: count, dtype: int64
housing
yes        4951
no         4104
unknown     225
Name: count, dtype: int64
loan
no         7688
yes        1367
unknown     225
Name: count, dtype: int64
month
may    2533
jul    1477
aug    1353
jun    1169
nov     886
apr     785
oct     369
mar     313
sep     295
dec

In [23]:
df = df.astype({'marital': 'category',
                                          'job': 'category',
                                          'education': 'category',
                                          'default': 'category',
                                          'housing': 'category',
                                          'loan': 'category',
                                          'month': 'category',
                                          'day_of_week': 'category',
                                          'contact': 'category',
                                          'campaign': 'category',
                                          'previous': 'category',
                                          'poutcome': 'category'})
df.dtypes

,0
age,int64
job,category
marital,category
education,category
default,category
housing,category
loan,category
contact,category
month,category
day_of_week,category


In [26]:
df_encoded_X = pd.get_dummies(df)

df_encoded_X.head()

,age,duration,pdays,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,job_admin.,job_blue-collar,...,previous_0,previous_1,previous_2,previous_3,previous_4,previous_5,previous_6,poutcome_failure,poutcome_nonexistent,poutcome_success
0,41,1575,999,1.1,93.994,-36.4,4.857,5191.0,False,True,...,True,False,False,False,False,False,False,False,True,False
1,49,1042,999,1.1,93.994,-36.4,4.857,5191.0,False,False,...,True,False,False,False,False,False,False,False,True,False
2,49,1467,999,1.1,93.994,-36.4,4.857,5191.0,False,False,...,True,False,False,False,False,False,False,False,True,False
3,41,579,999,1.1,93.994,-36.4,4.857,5191.0,False,False,...,True,False,False,False,False,False,False,False,True,False
4,45,461,999,1.1,93.994,-36.4,4.857,5191.0,False,True,...,True,False,False,False,False,False,False,False,True,False


In [27]:
y_target_encoded = pd.get_dummies(y_target)

y_target_encoded.head()

,no,yes
0,False,True
1,False,True
2,False,True
3,False,True
4,False,True


#Chunk 4:
###Train and Evaluate Simple Decision Tree 1 to classify y